In [171]:
import pandas as pd
import numpy as np
from scipy import optimize

In [172]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103904 entries, 0 to 103903
Data columns (total 25 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   Unnamed: 0                         103904 non-null  int64  
 1   id                                 103904 non-null  int64  
 2   Gender                             103904 non-null  object 
 3   Customer Type                      103904 non-null  object 
 4   Age                                103904 non-null  int64  
 5   Type of Travel                     103904 non-null  object 
 6   Class                              103904 non-null  object 
 7   Flight Distance                    103904 non-null  int64  
 8   Inflight wifi service              103904 non-null  int64  
 9   Departure/Arrival time convenient  103904 non-null  int64  
 10  Ease of Online booking             103904 non-null  int64  
 11  Gate location                      1039

In [173]:
#Preprocesamiento de datos
to_convert = ['Gender', 'Customer Type', 'Type of Travel', 'Class', 'satisfaction']
id2label = {}
label2id = {}
for col in to_convert:
    id2label[col] = {i: cls for i, cls in enumerate(train_data[col].unique())}
    label2id[col] = {cls: i for i, cls in id2label[col].items()}
    train_data[col] = train_data[col].replace(label2id[col])
    test_data[col] = test_data[col].replace(label2id[col])
    print(id2label[col])

num_labels = 2

{0: 'Male', 1: 'Female'}
{0: 'Loyal Customer', 1: 'disloyal Customer'}
{0: 'Personal Travel', 1: 'Business travel'}
{0: 'Eco Plus', 1: 'Business', 2: 'Eco'}
{0: 'neutral or dissatisfied', 1: 'satisfied'}


C:\Users\Hiro\AppData\Local\Temp\ipykernel_13544\2768912797.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_data[col] = train_data[col].replace(label2id[col])
C:\Users\Hiro\AppData\Local\Temp\ipykernel_13544\2768912797.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_data[col] = test_data[col].replace(label2id[col])


In [174]:
train_data, test_data = train_data.to_numpy(), test_data.to_numpy()
np.random.shuffle(train_data)  
x_train, y_train = train_data[:, 2:24], train_data[:, 24].astype(np.int64)
x_test, y_test = test_data[:, 2:24], test_data[:, 24].astype(np.int64)

print(x_train.shape, x_test.shape)

(103904, 22) (25976, 22)


In [175]:
#Normalización de los datos
def featureNormalize(x):
    mu = np.mean(x, axis=0)
    sigma = np.std(x, axis=0)
    x_norm = (x - mu) / sigma
    return x_norm

x_train = featureNormalize(x_train)
x_test = featureNormalize(x_test)

In [176]:
def randInitializeWeights(L_in, L_out, epsilon_init=0.12):
    W = np.zeros((L_out, 1 + L_in))
    W = np.random.rand(L_out, 1 + L_in) * 2 * epsilon_init - epsilon_init
    return W

In [177]:
input_layer_size  = 22
hidden_layer_size = 15
num_labels = 2

initial_Theta1 = randInitializeWeights(input_layer_size, hidden_layer_size)
initial_Theta2 = randInitializeWeights(hidden_layer_size, num_labels)

initial_nn_params = np.concatenate([initial_Theta1.ravel(), initial_Theta2.ravel()], axis=0)

In [178]:
def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

def sigmoidGradient(z):
    g = np.zeros(z.shape)
    g = sigmoid(z) * (1 - sigmoid(z))
    return g

In [179]:
def nnCostFunction(nn_params, input_layer_size, hidden_layer_size, num_labels, x, y, lambda_=0.0):

    Theta1 = np.reshape(nn_params[:hidden_layer_size * (input_layer_size + 1)],
                        (hidden_layer_size, (input_layer_size + 1)))

    Theta2 = np.reshape(nn_params[(hidden_layer_size * (input_layer_size + 1)):],
                        (num_labels, (hidden_layer_size + 1)))

    Theta1_grad = np.zeros(Theta1.shape)
    Theta2_grad = np.zeros(Theta2.shape)

    m = y.size

    a1 = np.concatenate([np.ones((m, 1)), x], axis=1)
    a2 = sigmoid(a1.dot(Theta1.T))
    a2 = np.concatenate([np.ones((a2.shape[0], 1)), a2], axis=1)
    a3 = sigmoid(a2.dot(Theta2.T))

    y_matrix = y.reshape(-1)
    y_matrix = np.eye(num_labels)[y_matrix]
  
    temp1 = Theta1
    temp2 = Theta2
   
    
    reg_term = (lambda_ / (2 * m)) * (np.sum(np.square(temp1[:, 1:])) + np.sum(np.square(temp2[:, 1:])))
    J = (-1 / m) * np.sum((np.log(a3) * y_matrix) + np.log(1 - a3) * (1 - y_matrix)) + reg_term

    delta_3 = a3 - y_matrix
    delta_2 = delta_3.dot(Theta2)[:, 1:] * sigmoidGradient(a1.dot(Theta1.T))
    Delta1 = delta_2.T.dot(a1)
    Delta2 = delta_3.T.dot(a2)

    Theta1_grad = (1 / m) * Delta1
    Theta1_grad[:, 1:] = Theta1_grad[:, 1:] + (lambda_ / m) * Theta1[:, 1:]
    Theta2_grad = (1 / m) * Delta2
    Theta2_grad[:, 1:] = Theta2_grad[:, 1:] + (lambda_ / m) * Theta2[:, 1:]

    grad = np.concatenate([Theta1_grad.ravel(), Theta2_grad.ravel()])

    return J, grad

In [180]:
options= {'maxiter': 1000}
lambda_ = 0.2
costFunction = lambda p: nnCostFunction(p, input_layer_size,
                                        hidden_layer_size,
                                        num_labels, x_train, y_train, lambda_)

res = optimize.minimize(costFunction,
                        initial_nn_params,
                        jac=True,
                        method='TNC',
                        options=options)

nn_params = res.x

Theta1 = np.reshape(nn_params[:hidden_layer_size * (input_layer_size + 1)],
                    (hidden_layer_size, (input_layer_size + 1)))
Theta2 = np.reshape(nn_params[(hidden_layer_size * (input_layer_size + 1)):],
                    (num_labels, (hidden_layer_size + 1)))

C:\Users\Hiro\AppData\Local\Temp\ipykernel_13544\4128511236.py:7: OptimizeWarning: Unknown solver options: maxiter
  res = optimize.minimize(costFunction,


In [ ]:
def predict(Theta1, Theta2, X):
    m = X.shape[0]
    p = np.zeros(m)
    h1 = sigmoid(np.dot(np.concatenate([np.ones((m, 1)), X], axis=1), Theta1.T))
    h2 = sigmoid(np.dot(np.concatenate([np.ones((m, 1)), h1], axis=1), Theta2.T))
    p = np.argmax(h2, axis=1)
    return p

In [ ]:
pred = predict(Theta1, Theta2, x_test)
print('Precision del conjunto de entrenamiento: {:.2f}%'.format(np.mean(pred == y_test) * 100))
pred_sample = predict(Theta1, Theta2, x_test[:100, :])
print(pred_sample)
print(y_test[:100])

Training Set Accuracy: 43.333269
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 1 0 1 1 0 1 0 1 0 1 1 0 1 1 0 1 1 0 0 1 1 1 0 1 1 0 1 1 0 1 1 1 0 1 1 1
 1 1 1 1 0 1 0 1 0 1 0 1 0 0 1 1 0 0 1 0 1 0 0 1 0 1 0 0 1 1 0 1 1 1 0 1 0
 0 0 0 1 0 0 0 1 0 0 1 0 1 1 1 1 0 1 1 0 1 0 0 1 0 1]
